In [1]:
import os
import numpy as np
import tifffile
import PIL
from PIL import Image, ImageSequence
from ScanImageTiffReader import ScanImageTiffReader, ScanImageTiffReaderContext
import imagej
import pandas as pd
import skimage
import shutil

In [2]:
def get_file_name(path,kind):  
    os.chdir(path)
    files=[]
    for file in os.listdir():
        if file.endswith(kind):
            files.append(file)
    return files
def get_pos(image_file_name):
    start=image_file_name.find('_')+1
    end=image_file_name.find('_',4)
    return image_file_name[start:end]
def get_row(image_file_name):
    start=image_file_name.find('_',10)+1
    end=start+3
    return int(image_file_name[start:end])

def get_col(image_file_name):
    start=image_file_name.find('_',7)+1
    end=start+3
    return int(image_file_name[start:end])

def reverse_both_row_col_name(file,Max_row,Max_col,pos_name):
    for i in range(len(pos_name)):
        batch=[name for name in file if pos_name[i] in name]
        for name in batch:
            new_r=Max_row[i]-get_row(name) #reverse row number
            new_c=Max_col[i]-get_col(name)
            row_str=str(new_r).zfill(3)
            col_str=str(new_c).zfill(3)
            new_name=name[0:name.find('P')]+pos_name[i]+'_'+col_str+'_'+ row_str+'.tif'#name[0:name.find('0')]='MAX_Pos1_'
            src_dir=os.path.join(pos_path,image_folder,name)
            dst_dir=os.path.join(pos_path,image_folder+'_fixedname',new_name)
            shutil.copy(src_dir,dst_dir)

def reverse_row_name(file,Max_row,pos_name):
    for i in range(len(pos_name)):
        batch=[name for name in file if pos_name[i] in name]
        for name in batch:
            new_r=Max_row[i]-get_row(name) #reverse row number
            new_c=get_col(name)
            row_str=str(new_r).zfill(3)
            col_str=str(new_c).zfill(3)
            new_name=name[0:name.find('P')]+pos_name[i]+'_'+col_str+'_'+ row_str+'.tif'#name[0:name.find('0')]='MAX_Pos1_'
            src_dir=os.path.join(pos_path,image_folder,name)
            dst_dir=os.path.join(pos_path,image_folder+'_fixedname',new_name)
            shutil.copy(src_dir,dst_dir)
            
def reverse_col_name(file,Max_row,pos_name):
    for i in range(len(pos_name)):
        batch=[name for name in file if pos_name[i] in name]
        for name in batch:
            new_r=get_row(name) #reverse row number
            new_c=Max_col[i]-get_col(name)
            row_str=str(new_r).zfill(3)
            col_str=str(new_c).zfill(3)
            new_name=name[0:name.find('P')]+pos_name[i]+'_'+col_str+'_'+ row_str+'.tif'#name[0:name.find('0')]='MAX_Pos1_'
            src_dir=os.path.join(pos_path,image_folder,name)
            dst_dir=os.path.join(pos_path,image_folder+'_fixedname',new_name)
            shutil.copy(src_dir,dst_dir)
            
            
def get_global_trans_matrix(pos):
    img_names = list()
    pixel_x_position = list()
    pixel_y_position = list()
    with open(global_positions_filepath, 'r') as fh:
        for line in fh:
            line = line.strip()
            toks = line.split(';')

            # name loading
            fn_tok = toks[0]
            fn = fn_tok.split(':')[1].strip()
            img_names.append(fn)

            # position loading
            pos_tok = toks[2]
            pos_pair = pos_tok.split(':')[1].strip()
            pos_pair = pos_pair.replace(')', '')
            pos_pair = pos_pair.replace('(', '')
            pos_pairs = pos_pair.split(',')
            x = int(pos_pairs[0].strip())
            y = int(pos_pairs[1].strip())
            pixel_x_position.append(x)
            pixel_y_position.append(y)
    d={'img_names':img_names,'pixel_x_position':pixel_x_position,'pixel_y_position':pixel_y_position}
    trans = pd.DataFrame(data=d)
    trans.to_csv(os.path.join(pos_path,target_channel,'stitched_'+pos,pos+'_transformation.csv'), index=False)
    return img_names,pixel_x_position,pixel_y_position

def create_empty_stitched_img(images_dirpath,img_names):
    first_tile = skimage.io.imread(os.path.join(images_dirpath, img_names[0]))
    tile_shape = first_tile.shape
    n_channels = 1
    if len(tile_shape) == 3:
        n_channels = tile_shape[2]
    tile_h = tile_shape[0]
    tile_w = tile_shape[1]

    stitched_img_h = tile_h + np.max(pixel_y_position)
    stitched_img_w = tile_w + np.max(pixel_x_position)
    stitched_img = np.zeros((stitched_img_h, stitched_img_w), dtype=first_tile.dtype)
    return stitched_img,tile_shape,first_tile,tile_h,tile_w

## presetting

In [3]:
pos_path=os.path.join("E:\\Allenwork\\Project\\","stitch_image_with_MIST","20230911_post_xenium_ZM")
image_folder='hyb01'
stitch_image_folder=image_folder

file=get_file_name(os.path.join(pos_path,image_folder),'.tif')
channal_number=6
c_ls=["ch1","ch2","ch3","ch4","ch5","ch6"]
file

['MAX_Pos1_000_000.tif',
 'MAX_Pos1_000_001.tif',
 'MAX_Pos1_000_002.tif',
 'MAX_Pos1_000_003.tif',
 'MAX_Pos1_000_004.tif',
 'MAX_Pos1_000_005.tif',
 'MAX_Pos1_000_006.tif',
 'MAX_Pos1_000_007.tif',
 'MAX_Pos1_000_008.tif',
 'MAX_Pos1_000_009.tif',
 'MAX_Pos1_001_000.tif',
 'MAX_Pos1_001_001.tif',
 'MAX_Pos1_001_002.tif',
 'MAX_Pos1_001_003.tif',
 'MAX_Pos1_001_004.tif',
 'MAX_Pos1_001_005.tif',
 'MAX_Pos1_001_006.tif',
 'MAX_Pos1_001_007.tif',
 'MAX_Pos1_001_008.tif',
 'MAX_Pos1_001_009.tif',
 'MAX_Pos1_002_000.tif',
 'MAX_Pos1_002_001.tif',
 'MAX_Pos1_002_002.tif',
 'MAX_Pos1_002_003.tif',
 'MAX_Pos1_002_004.tif',
 'MAX_Pos1_002_005.tif',
 'MAX_Pos1_002_006.tif',
 'MAX_Pos1_002_007.tif',
 'MAX_Pos1_002_008.tif',
 'MAX_Pos1_002_009.tif',
 'MAX_Pos1_003_000.tif',
 'MAX_Pos1_003_001.tif',
 'MAX_Pos1_003_002.tif',
 'MAX_Pos1_003_003.tif',
 'MAX_Pos1_003_004.tif',
 'MAX_Pos1_003_005.tif',
 'MAX_Pos1_003_006.tif',
 'MAX_Pos1_003_007.tif',
 'MAX_Pos1_003_008.tif',
 'MAX_Pos1_003_009.tif',


In [4]:
Max_row=[]
Max_col=[]
pos=[1]
pos_name=['Pos'+str(n) for n in pos]
for i in pos_name:
    n1=max([get_row(name) for name in file if i in name])
    n2=max([get_col(name) for name in file if i in name])
    Max_row.append(n1)
    Max_col.append(n2)

## rename the image (optional, in case the file name row and col reversed )

In [7]:
##rename the image (optional, in case the file name row and col reversed )
os.mkdir(os.path.join(pos_path,image_folder+'_fixedname'))
reverse_row_name(file,Max_row,pos_name)
#reverse_col_name(file,Max_col,pos_name)
#reverse_both_row_col_name(file,Max_row,Max_col,pos_name)
stitch_image_folder=image_folder+'_fixedname'

# Arrange folders  (Ignore here, it is just for our pipeline)

In [8]:
trim=0.02
file=get_file_name(os.path.join(pos_path,stitch_image_folder),'.tif')
img=tifffile.imread(os.path.join(pos_path,stitch_image_folder,file[0]))
Img_w=img.shape[1]
Img_h=img.shape[1]
stitch_image_folder

'hyb01_fixedname'

In [9]:
#arrange folders


file=get_file_name(os.path.join(pos_path,stitch_image_folder),'.tif')
pos=[get_pos(f) for f in file]
pos_ls=np.unique(pos)
c_ls=["ch1","ch2","ch3","ch4","ch5","ch6"]
for i in c_ls:
    if not os.path.exists(os.path.join(pos_path,i)):
        os.mkdir(os.path.join(pos_path,i))        
for i in c_ls:
    for j in pos_ls:
        if not os.path.exists(os.path.join(pos_path,i,j)):
            os.mkdir(os.path.join(pos_path,i,j)) 
        if not os.path.exists(os.path.join(pos_path,i,"stitched_"+j)):
            os.mkdir(os.path.join(pos_path,i,"stitched_"+j))
pos_index_ls=[]
for pos_name in pos_ls:
    index=[i for i, x in enumerate(pos) if x == pos_name]
    pos_index_ls.append(index)
for i in range(len(pos_index_ls)):
    folder=pos_ls[i]
    name=[file[index] for index in pos_index_ls[i]]
    for n in name:
        img=tifffile.imread(os.path.join(pos_path,stitch_image_folder,n))
        img_crop=img[:,int(Img_w*trim):int(Img_w-(Img_w*trim)),int(Img_h*trim):int(Img_h-(Img_h*trim))]
        for j in range(len(img)):
            img_single=Image.fromarray(img_crop[j])
            
            img_single.save(os.path.join(pos_path,c_ls[j],folder,n))
            print(os.path.join(pos_path,c_ls[j],folder,n))

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_000_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_000_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_000_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_000_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_000_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_000_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_000_001.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_000_001.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_000_001.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_000_001.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_001_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_001_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_001_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_001_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_001_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_001_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_001_005.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_001_005.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_001_005.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_001_005.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_002_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_002_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_002_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_002_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_002_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_002_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_002_009.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_002_009.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_002_009.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_002_009.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_004_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_004_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_004_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_004_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_004_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_004_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_004_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_004_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_004_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_004_003.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_005_006.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_005_006.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_005_006.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_005_006.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_005_006.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_005_006.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_005_007.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_005_007.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_005_007.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_005_007.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_007_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_007_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_007_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_007_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_007_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_007_000.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_007_001.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_007_001.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_007_001.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_007_001.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_008_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_008_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_008_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_008_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_008_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_008_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_008_005.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_008_005.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_008_005.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_008_005.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_009_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_009_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_009_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_009_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_009_008.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_009_009.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_009_009.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_009_009.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_009_009.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_009_009.tif
E:\Allenwork\Project

E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_011_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_011_002.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_011_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_011_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch3\Pos1\MAX_Pos1_011_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\Pos1\MAX_Pos1_011_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch5\Pos1\MAX_Pos1_011_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\Pos1\MAX_Pos1_011_003.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\Pos1\MAX_Pos1_011_004.tif
E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\Pos1\MAX_Pos1_011_004.tif
E:\Allenwork\Project

# config with imageJ

In [10]:
ij=imagej.init('E:\\Allenwork\\Software\\Fiji.app')

In [11]:
ij.getVersion()

'2.14.0/1.54f'

# stitch with MIST in ImageJ

In [13]:


for stitch_channal in c_ls:
    for i in pos_ls:
        image_file_name_ls=os.listdir(os.path.join(pos_path,stitch_channal,i))
        image_file_name_ls=os.listdir(os.path.join(pos_path,stitch_channal,i))
        row=[get_row(i) for i in image_file_name_ls]
        col=[get_col(i) for i in image_file_name_ls]
        Height=int(max(row)+1)
        Width=int(max(col)+1)
        Starting_point="Upper Right"
        img_directory=os.path.join(pos_path,stitch_channal,i)
        img_save_directory=os.path.join(pos_path,stitch_channal,"stitched_"+i)
        filenamePattern='Max_'+i+'_\{ccc\}_\{rrr\}.tif'
        img_directory = img_directory.replace('\\','\\\\')
        img_save_directory=img_save_directory.replace('\\','\\\\')
        macro = f'''run("MIST",\
        "gridwidth={Width} \
        gridheight={Height} \
        starttilerow=0 \
        starttilecol=0 \
        imagedir={img_directory} \
        filenamepattern={filenamePattern} \
        filenamepatterntype=ROWCOL \
        gridorigin=UR \
        assemblefrommetadata=false \
        assemblenooverlap=false \
        globalpositionsfile=[] \
        numberingpattern=HORIZONTALCOMBING \
        startrow=0 \
        startcol=0 \
        extentwidth={Width} \
        extentheight={Height} \
        timeslices=0 \
        istimeslicesenabled=false \
        outputpath={img_save_directory} \
        displaystitching=true outputfullimage=true outputmeta=true outputimgpyramid=false \
        blendingmode=AVERAGE blendingalpha=NaN compressionmode=UNCOMPRESSED outfileprefix=img- \
        unit=MICROMETER unitx=1.0 unity=1.0 programtype=JAVA numcputhreads=8 loadfftwplan=true \
        savefftwplan=true fftwplantype=MEASURE fftwlibraryname=libfftw3 \
        fftwlibraryfilename=libfftw3.dll planpath=C:\\\\Fiji.app\\\\lib\\\\fftw\\\\fftPlans \
        fftwlibrarypath=C:\\\\Fiji.app\\\\lib\\\\fftw stagerepeatability=0 \
        horizontaloverlap=20.0 \
        verticaloverlap=20.0 \
        numfftpeaks=0 \
        overlapuncertainty=NaN \
        isusedoubleprecision=false \
        isusebioformats=false \
        issuppressmodelwarningdialog=false \
        isenablecudaexceptions=false \
        translationrefinementmethod=SINGLE_HILL_CLIMB \
        numtranslationrefinementstartpoints=16 \
        headless=false \
        loglevel=MANDATORY \
        debuglevel=NONE");'''
        ij.py.run_macro(macro)

[java.lang.Enum.toString] Warning: the following files will be overwritten:[java.lang.Enum.toString] 
[java.lang.Enum.toString] E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\stitched_Pos1\img-global-positions-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\stitched_Pos1\img-log.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\stitched_Pos1\img-relative-positions-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\stitched_Pos1\img-relative-positions-no-optimization-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch1\stitched_Pos1\img-statistics.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] E:\Allenwork\Project\stitch_image_with_MIST\202309

[java.lang.Enum.toString] Completed Stitching in 28140ms[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing global positions to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\stitched_Pos1\img-global-positions-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing relative positions to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\stitched_Pos1\img-relative-positions-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing relative positions (no optimization) to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\stitched_Pos1\img-relative-positions-no-optimization-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing full image to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch2\stitched_Pos1\img-stitched-0.ome.tif  Width: 30120 Height: 25196[java.lang.Enum.toString] 
[java.lang.Enum.toString] Blending Profile: Init Time: 51140 Blend Time: 4079 Ble

[java.lang.Enum.toString] Writing relative positions (no optimization) to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch4\stitched_Pos1\img-relative-positions-no-optimization-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computed North overlap: 20.0%[java.lang.Enum.toString] 
[java.lang.Enum.toString] Computed West overlap: 20.0%[java.lang.Enum.toString] 
[java.lang.Enum.toString] Warning: no good translations found for North direction. Estimated translations generated from the overlap.[java.lang.Enum.toString] 
[java.lang.Enum.toString] Warning: no good translations found for North direction. Repeatability has been set to zero.[java.lang.Enum.toString] 
[java.lang.Enum.toString] Please check the statistics file for more details.[java.lang.Enum.toString] 
[java.lang.Enum.toString] Repeatability for North: 0 pixels[java.lang.Enum.toString] 
[java.lang.Enum.toString] Repeatability for West: 0 pixels[java.lang.Enum.toString] 
[java.lang.Enum.toString] 

[java.lang.Enum.toString] Completed Stitching in 32479ms[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing global positions to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\stitched_Pos1\img-global-positions-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing relative positions to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\stitched_Pos1\img-relative-positions-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing relative positions (no optimization) to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\stitched_Pos1\img-relative-positions-no-optimization-0.txt[java.lang.Enum.toString] 
[java.lang.Enum.toString] Writing full image to: E:\Allenwork\Project\stitch_image_with_MIST\20230911_post_xenium_ZM\ch6\stitched_Pos1\img-stitched-0.ome.tif  Width: 30152 Height: 25228[java.lang.Enum.toString] 
[java.lang.Enum.toString] Blending Profile: Init Time: 51363 Blend Time: 4259 Ble

# get the stitch matrix and apply to each channel

In [52]:
global_positions_filepath=os.path.join(pos_path,stitch_channal,'stitched_Pos1','img-global-positions-0.txt')
target_channel='ch6'
images_dirpath=os.path.join(pos_path,target_channel,'Pos1')
stitch_images_dirpath=os.path.join(pos_path,target_channel,'Pos1')
stitch_images_savedirpath=os.path.join(pos_path,target_channel,'stitched_Pos1')

In [63]:
for ch in c_ls:
    for pos in pos_ls:
        global_positions_filepath=os.path.join(pos_path,stitch_channal,'stitched_'+pos,'img-global-positions-0.txt')
        target_channel=ch
        images_dirpath=os.path.join(pos_path,target_channel,pos)
        stitch_images_dirpath=os.path.join(pos_path,target_channel,pos)
        stitch_images_savedirpath=os.path.join(pos_path,target_channel,'stitched_'+pos)
        img_names,pixel_x_position,pixel_y_position=get_global_trans_matrix(pos)
        stitched_img,tile_shape,first_tile,tile_h,tile_w=create_empty_stitched_img(images_dirpath,img_names)
        for i in range(0, len(img_names)):
            fn = img_names[i]
            x = pixel_x_position[i]
            y = pixel_y_position[i]

            tile = skimage.io.imread(os.path.join(stitch_images_dirpath, fn))
            if tile.shape != tile_shape:
                raise RuntimeError('All images must be the same shape. Image {} is {}, expected {}'.format(fn, tile.shape, tile_shape))
            if tile.dtype != first_tile.dtype:
                raise RuntimeError('Img {} has type: {}, expected {}.'.format(fn, tile.dtype, first_tile.dtype))

            stitched_img[y:y+tile_h, x:x+tile_w] = tile
        skimage.io.imsave(os.path.join(stitch_images_savedirpath,'stitched_'+pos+'.tif'),stitched_img,photometric='minisblack')

C:\Users\aixin.zhang\AppData\Local\Temp\ipykernel_17644\1400173055.py:22: UserWarning: E:\Allenwork\Project\stitch_image_with_MIST\ch1\stitched_Pos1\stitched_Pos1.tif is a low contrast image
  skimage.io.imsave(os.path.join(stitch_images_savedirpath,'stitched_'+pos+'.tif'),stitched_img,photometric='minisblack')
C:\Users\aixin.zhang\AppData\Local\Temp\ipykernel_17644\1400173055.py:22: UserWarning: E:\Allenwork\Project\stitch_image_with_MIST\ch1\stitched_Pos2\stitched_Pos2.tif is a low contrast image
  skimage.io.imsave(os.path.join(stitch_images_savedirpath,'stitched_'+pos+'.tif'),stitched_img,photometric='minisblack')
C:\Users\aixin.zhang\AppData\Local\Temp\ipykernel_17644\1400173055.py:22: UserWarning: E:\Allenwork\Project\stitch_image_with_MIST\ch1\stitched_Pos3\stitched_Pos3.tif is a low contrast image
  skimage.io.imsave(os.path.join(stitch_images_savedirpath,'stitched_'+pos+'.tif'),stitched_img,photometric='minisblack')
C:\Users\aixin.zhang\AppData\Local\Temp\ipykernel_17644\14001